<a href="https://colab.research.google.com/github/ridhi-png/Financial-News-Sentiment-Analyzer/blob/main/Financial%20News%20Sentiment%20.%20ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install streamlit transformers torch scipy pandas pyngrok


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 52.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 62.2 MB/s eta 0:00:00


In [2]:
%%writefile sentiment_model.py
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from scipy.special import softmax

model_name = "ProsusAI/finbert"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)

def analyze_sentiment(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True)
    outputs = model(**inputs)
    scores = outputs[0][0].detach().numpy()
    scores = softmax(scores)

    labels = ['negative', 'neutral', 'positive']
    result = {labels[i]: float(scores[i]) for i in range(3)}
    sentiment = max(result, key=result.get)
    return sentiment, result


Writing sentiment_model.py


In [3]:
%%writefile app.py
import streamlit as st
import pandas as pd
from sentiment_model import analyze_sentiment

st.set_page_config(page_title="Financial News Sentiment Analyzer", page_icon="🧠", layout="centered")
st.title("🧠 Financial News Sentiment Analyzer")
st.write("Analyze finance-related headlines using AI (FinBERT).")

headline = st.text_area("Enter a financial headline or short news snippet:",
                        "Apple shares rose after strong Q4 earnings report")

if st.button("🔍 Analyze Sentiment"):
    sentiment, result = analyze_sentiment(headline)
    st.subheader(f"Sentiment: **{sentiment.upper()}**")
    st.write(result)

st.markdown("---")
st.header("📁 Analyze Multiple Headlines (CSV Upload)")
uploaded_file = st.file_uploader("Upload a CSV file with a 'headline' column")

if uploaded_file is not None:
    df = pd.read_csv(uploaded_file)
    sentiments = []
    for text in df['headline']:
        s, _ = analyze_sentiment(text)
        sentiments.append(s)
    df['Sentiment'] = sentiments
    st.write(df)
    st.download_button("⬇️ Download Results", df.to_csv(index=False), "results.csv", "text/csv")


Writing app.py


In [4]:
%%writefile sample_news.csv
headline
Apple stock jumps after quarterly earnings beat expectations
Tesla faces production delays amid supply chain issues
Federal Reserve hints at possible rate cuts next quarter
Amazon launches new AI-driven warehouse system
Oil prices drop as global demand weakens


Writing sample_news.csv


In [5]:
!npm install -g cloudflared


⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸
added 1 package in 3s
⠸

In [6]:
!pip install gradio transformers torch scipy

from transformers import AutoTokenizer, AutoModelForSequenceClassification
from scipy.special import softmax
import gradio as gr

model_name = "ProsusAI/finbert"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)

def analyze_sentiment(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True)
    outputs = model(**inputs)
    scores = outputs[0][0].detach().numpy()
    scores = softmax(scores)
    labels = ['negative', 'neutral', 'positive']
    return dict(zip(labels, map(float, scores)))

demo = gr.Interface(fn=analyze_sentiment,
                    inputs="text",
                    outputs="label",
                    title="🧠 Financial Sentiment Analyzer (FinBERT)")
demo.launch(share=True)


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/252 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/758 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/438M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/438M [00:00<?, ?B/s]

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://a0034571dec3347a2e.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [7]:
from transformers import AutoModelForSequenceClassification
model = AutoModelForSequenceClassification.from_pretrained("ProsusAI/finbert")

print(model.config.id2label)


{0: 'positive', 1: 'negative', 2: 'neutral'}


In [8]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from scipy.special import softmax

model_name = "ProsusAI/finbert"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)

# ✅ Correct order according to model.config.id2label
labels = ['positive', 'negative', 'neutral']

def analyze_sentiment(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True)
    outputs = model(**inputs)
    scores = outputs.logits[0].detach().numpy()
    scores = softmax(scores)
    sentiment = dict(zip(labels, map(float, scores)))
    return sentiment


In [11]:
# Install (run once in Colab)
!pip install -q gradio transformers torch scipy

# Code
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from scipy.special import softmax
import gradio as gr

MODEL_NAME = "ProsusAI/finbert"  # ensure this is the model you're using

print("Loading model... (this may take ~20-60s on first run)")
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModelForSequenceClassification.from_pretrained(MODEL_NAME)
print("Model loaded.")

# --- Get the model's label mapping reliably ---
raw_map = model.config.id2label  # e.g. {0: 'positive', 1: 'negative', 2: 'neutral'}
# Convert to dict of index -> label, and build ordered list
label_order = [raw_map[i] for i in sorted(raw_map.keys())]
print("Detected label mapping (index -> label):")
for i, lab in enumerate(label_order):
    print(f"  {i} -> {lab}")

def analyze_text(text):
    """
    Returns (label-dict, label-dict) so Gradio can show a Label widget and JSON.
    label-dict = { 'positive': 0.87, 'negative': 0.05, 'neutral': 0.08 }
    """
    if not isinstance(text, str) or text.strip() == "":
        return {}, {}
    inputs = tokenizer(text, return_tensors="pt", truncation=True)
    outputs = model(**inputs)
    logits = outputs.logits[0].detach().numpy()
    probs = softmax(logits)
    # Build dict using the detected order
    probs_dict = { label_order[i]: float(probs[i]) for i in range(len(probs)) }
    # Gradio's Label component expects a dict label->score
    return probs_dict, probs_dict

# --- Quick local tests (prints to notebook so you can verify) ---
examples = [
    "Apple shares rise after record quarterly profits",
    "Tesla faces production delays due to chip shortage",
    "Federal Reserve maintains current interest rates",
    "Amazon reports disappointing sales this quarter"
]

print("\nRunning quick tests:")
for ex in examples:
    out, _ = analyze_text(ex)
    # Determine top label
    if out:
        top = max(out, key=out.get)
        print(f"  Input: {ex!s}")
        print(f"   -> top: {top} | probs: {out}")
    else:
        print("  Skipped empty input.")

# --- Gradio UI ---
label_component = gr.Label(num_top_classes=3)  # shows top classes and probabilities
json_component = gr.JSON()

demo = gr.Interface(
    fn=analyze_text,
    inputs=gr.Textbox(lines=3, placeholder="Type a financial headline here..."),
    outputs=[label_component, json_component],
    title="Financial News Sentiment (FinBERT)",
    description="Paste a financial headline (e.g., 'Apple shares rise after record quarterly profits') and this model will return Positive/Negative/Neutral probabilities."
)

# Launch with a public share link (works in Colab)
demo.launch(share=True)


Loading model... (this may take ~20-60s on first run)
Model loaded.
Detected label mapping (index -> label):
  0 -> positive
  1 -> negative
  2 -> neutral

Running quick tests:
  Input: Apple shares rise after record quarterly profits
   -> top: positive | probs: {'positive': 0.6375828981399536, 'negative': 0.2820237874984741, 'neutral': 0.08039329946041107}
  Input: Tesla faces production delays due to chip shortage
   -> top: negative | probs: {'positive': 0.008175628259778023, 'negative': 0.9496684074401855, 'neutral': 0.04215594008564949}
  Input: Federal Reserve maintains current interest rates
   -> top: neutral | probs: {'positive': 0.04541559889912605, 'negative': 0.03632590174674988, 'neutral': 0.9182584881782532}
  Input: Amazon reports disappointing sales this quarter
   -> top: negative | probs: {'positive': 0.0100867273285985, 'negative': 0.9735027551651001, 'neutral': 0.0164104625582695}
Colab notebook detected. To show errors in colab notebook, set debug=True in launch(